# Converting SCEN7 files to SCEN

Hacky notebook which may have some useful bits but shouldn't ever appear in master of pymagicc in this form.

In [1]:
%matplotlib inline

from os import listdir
from os.path import join, dirname
from pprint import pprint
from copy import deepcopy

import pandas as pd
import numpy as np
import pyam
from pyam.utils import LONG_IDX
import pint
from pint.pandas_interface import PintArray
from pint.errors import DimensionalityError

import pymagicc
from pymagicc.io import MAGICCData

import matplotlib.pyplot as plt
plt.style.use('bmh') 

import expectexception

<IPython.core.display.Javascript object>

In [2]:
scen7_path = "/Users/zebedeenicholls/Documents/AGCEC/UniMelb-Teaching/Climate-Modelling-and-Climate-Change-ERTH90026/notebooks/MAGICC7/run/"

In [30]:
def make_total_vars(mdata):
    variables = mdata.df.columns.get_level_values("VARIABLE")
    to_append = []
    for var in variables:
        pair_var = var.replace("I_", "B_")
        if (pair_var in variables) and (pair_var != var):
            var_df = mdata[var].copy()
            var_df.columns = var_df.columns.droplevel('VARIABLE')

            pair_var_df = mdata[pair_var].copy()
            pair_var_df.columns = pair_var_df.columns.droplevel('VARIABLE')

            sum_df = var_df + pair_var_df
            var_base = var.split("_")[0][:-1]
            total_var = "_".join([var_base, "EMIS"])
            sum_df = pd.concat([sum_df], keys=[total_var], names=["VARIABLE"], axis=1)
            to_append.append(sum_df)

    mdata.df = pd.concat([mdata.df] + to_append, axis=1)
    mdata.df   
    
    return mdata

def convert_scen7_to_scen_hack(scen_file_path, scen_file_name):
    mdata = MAGICCData()
    mdata.read(filepath=scen_file_path, filename=scen_file_name)
    
    mdatatwo = MAGICCData()
    mdatatwo.read(filepath=scen_file_path, filename=scen_file_name.replace("WORLD_Ext2250.SCEN7", "HFC_C2F6_CF4_SF6_Ext2250.SCEN7"))
    
    mdatathree = MAGICCData()
    mdatathree.read(filepath=scen_file_path, filename=scen_file_name.replace("WORLD_Ext2250.SCEN7", "HFC_C2F6_CF4_SF6_MISSGAS_Ext2250.SCEN7"))
    
    mdata.df = pd.concat([mdata.df, mdatatwo.df, mdatathree.df], axis=1)
    
    mdata = make_total_vars(mdata)

    expect_vars = [v + "_EMIS" for v in pymagicc.definitions.scen_emms_code_1]
    mdata.df = mdata.iloc[
        :, 
        mdata.columns.get_level_values("VARIABLE").isin(expect_vars)
    ].dropna().T.drop_duplicates().T

    mdata.write(scen_file_name.replace("SCEN7", "SCEN"), scen_file_path)

In [31]:
mdata = MAGICCData(scen7_path)
for f in listdir(scen7_path):
    if not f.endswith("WORLD_Ext2250.SCEN7"):
        continue
    convert_scen7_to_scen_hack(scen7_path, f)